In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import string

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['dev_server']
sladb = config['db']['sladb']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + sladb + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Write all of the sql queries required for the tests

In [5]:
sql_count_sla_id = 'select count(*) from sladb.dbo.tbl_ref_sla'

In [6]:
sql_distinct_sla_id = 'select distinct sla_id from sladb.dbo.tbl_ref_sla'

In [116]:
# sql_sla_code = '''select * from \
# sladb.dbo.tbl_ref_sla_code'''

In [115]:
sql_sla_code = '''select min(sla_code) as min_sla_code, 
                          max(sla_code) as max_sla_code, 
                          count(*) as n_sla_codes
                   from sladb.dbo.tbl_ref_sla_code'''

In [8]:
sql_date_category = 'select distinct(date_category_id) from sladb.dbo.tbl_ref_sla_season_category'

In [9]:
sql_sla_translation = 'select sla_id, sla_code, date_category_id from sladb.dbo.tbl_ref_sla_translation'

## Execute all of the sql queries and store the results

In [10]:
before_count_sla_id = pd.read_sql(con = engine, sql = sql_count_sla_id).values[0][0]

In [11]:
before_count_sla_id

6

In [12]:
before_squared_count_sla_id = before_count_sla_id * before_count_sla_id

In [13]:
before_squared_count_sla_id

36

In [14]:
before_distinct_sla_id = pd.read_sql(con = engine, sql = sql_distinct_sla_id)

In [15]:
before_distinct_sla_id

,sla_id
0,A
1,B
2,C
3,D
4,N
5,T


In [119]:
before_sla_code = pd.read_sql(con = engine, sql = sql_sla_code)

In [120]:
before_sla_code

,min_sla_code,max_sla_code,n_sla_codes
0,1,64,64


In [18]:
before_date_category = pd.read_sql(con = engine, sql = sql_date_category)

In [19]:
before_date_category

,date_category_id
0,1
1,2


In [20]:
before_sla_translation = pd.read_sql(con = engine, sql = sql_sla_translation)

In [21]:
before_sla_translation

,sla_id,sla_code,date_category_id
0,A,1,1
1,A,2,1
2,A,3,1
3,A,4,1
4,A,17,1
...,...,...,...
67,T,20,2
68,T,22,2
69,T,28,2
70,T,30,2


In [22]:
timing = []

In [23]:
descriptions = []

In [24]:
results = []

## Before Tests

### First before test

In [25]:
timing.append('before')

In [26]:
descriptions.append('''Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of 
                       sla_codes in tbl_ref_sla_code''')

In [27]:
results.append(before_squared_count_sla_id == before_sla_code['n_sla_codes'].values[0])

### Second before test

In [28]:
timing.append('before')

In [29]:
descriptions.append('''Test that the number of sla_codes in tbl_ref_sla_code is equal to the maximum sla_code
                       in tbl_ref_sla_code''')

In [30]:
results.append(before_sla_code['n_sla_codes'].values[0] == before_sla_code['max_sla_code'].values[0])

### Third before test

In [31]:
timing.append('before')

In [32]:
descriptions.append('''Test that the minimum sla_code in tbl_ref_sla_code is equal to 1''')

In [33]:
results.append(1 == before_sla_code['min_sla_code'].values[0])

### Fourth before test

In [34]:
timing.append('before')

In [35]:
descriptions.append('''Test that the count of records in tbl_ref_sla_translation is equal to the 
                       count of sla_codes in tbl_ref_sla_code multiplied by the number of date_category_ids 
                       in tbl_ref_sla_season_category''')

In [36]:
results.append(len(before_sla_translation) == (before_sla_code['n_sla_codes'].values[0] * len(before_date_category)))

### Fifth before test

In [37]:
timing.append('before')

In [38]:
descriptions.append('''Test the uniqueness of the rows in tbl_ref_sla_translation''')

In [39]:
before_sla_translation_nodup = before_sla_translation.drop_duplicates()

In [40]:
results.append(len(before_sla_translation) == len(before_sla_translation_nodup))

### Sixth before test

In [41]:
timing.append('before')

In [42]:
descriptions.append('''Test that all sla_ids, sla_codes and date_category_ids have been
                       accounted for in tbl_ref_sla_translation''')

In [43]:
before_unique_sla_translation = before_sla_translation.nunique()

In [44]:
results.append(((before_unique_sla_translation['sla_id'] == before_count_sla_id) 
                and (before_unique_sla_translation['date_category_id'] == len(before_date_category))
                and (before_unique_sla_translation['sla_code'] == before_sla_code['n_sla_codes'].values[0])))

In [45]:
list(zip(timing, descriptions, results))

[('before',
  'Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of \n                       sla_codes in tbl_ref_sla_code',
  True),
 ('before',
  'Test that the number of sla_codes in tbl_ref_sla_code is equal to the maximum sla_code\n                       in tbl_ref_sla_code',
  True),
 ('before',
  'Test that the minimum sla_code in tbl_ref_sla_code is equal to 1',
  True),
 ('before',
  'Test that the count of records in tbl_ref_sla_translation is equal to the \n                       count of sla_codes in tbl_ref_sla_code multiplied by the number of date_category_ids \n                       in tbl_ref_sla_season_category',
  True),
 ('before',
  'Test the uniqueness of the rows in tbl_ref_sla_translation',
  True),
 ('before',
  'Test that all sla_ids, sla_codes and date_category_ids have been\n                       accounted for in tbl_ref_sla_translation',
  True)]

### Insert the sla_id values into tbl_ref_sla so that the after tests can be conducted

In [46]:
ninserts = 2

In [47]:
sla_id = [x for x in string.ascii_uppercase if x not in(before_distinct_sla_id.values)]

In [48]:
sla_inserts = [[x, 'Testing new SLA: '+ x, 1, 2] for x in sla_id]

In [56]:
sla_inserts_df = pd.DataFrame(sla_inserts[0:ninserts], 
                              columns = ['sla_id', 'sla_desc', 'sla_min_days', 'sla_max_days'])

In [62]:
sla_inserts_df

,sla_id,sla_desc,sla_min_days,sla_max_days
0,E,Testing new SLA: E,1,2
1,F,Testing new SLA: F,1,2


In [63]:
sla_inserts_df.to_sql('tbl_ref_sla', con = engine, index = False, if_exists = 'append')

## Execute all of the sql queries AFTER INSERT and store the results

In [64]:
after_count_sla_id = pd.read_sql(con = engine, sql = sql_count_sla_id).values[0][0]

In [65]:
after_count_sla_id

8

In [68]:
after_squared_count_sla_id = (after_count_sla_id)**2 

In [69]:
after_squared_count_sla_id

64

In [70]:
after_distinct_sla_id = pd.read_sql(con = engine, sql = sql_distinct_sla_id)

In [71]:
after_distinct_sla_id

,sla_id
0,A
1,B
2,C
3,D
4,E
5,F
6,N
7,T


In [72]:
after_sla_code = pd.read_sql(con = engine, sql = sql_sla_code)

In [73]:
after_sla_code

,min_sla_code,max_sla_code,n_sla_codes
0,1,64,64


In [74]:
after_date_category = pd.read_sql(con = engine, sql = sql_date_category)

In [75]:
after_date_category

,date_category_id
0,1
1,2


In [76]:
after_sla_translation = pd.read_sql(con = engine, sql = sql_sla_translation)

In [77]:
after_sla_translation

,sla_id,sla_code,date_category_id
0,A,1,1
1,A,2,1
2,A,3,1
3,A,4,1
4,A,17,1
...,...,...,...
123,T,28,2
124,T,30,2
125,T,36,2
126,T,47,2


## After Tests

### First after test

In [51]:
timing.append('after')

In [78]:
descriptions.append('''Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of 
                       sla_codes in tbl_ref_sla_code''')

In [79]:
results.append(after_squared_count_sla_id == after_sla_code['n_sla_codes'].values[0])

In [101]:
results

[True, True, True, True, True, True, True, True, True, True, True, True]

### Second after test

In [82]:
timing.append('after')

In [83]:
descriptions.append('''Test that the number of sla_codes in tbl_ref_sla_code \
is equal to the maximum sla_code in tbl_ref_sla_code''')

In [84]:
results.append(after_sla_code['n_sla_codes'].values[0] == after_sla_code['max_sla_code'].values[0])

### Third after test

In [86]:
timing.append('after')

In [32]:
descriptions.append('''Test that the minimum sla_code in tbl_ref_sla_code is equal to 1''')

In [87]:
results.append(1 == after_sla_code['min_sla_code'].values[0])

### Fourth after test

In [89]:
timing.append('after')

In [90]:
descriptions.append(
    '''Test that the count of records in tbl_ref_sla_translation is \
    equal to the count of sla_codes in tbl_ref_sla_code multiplied by \
    the number of date_category_ids in tbl_ref_sla_season_category''')

In [91]:
results.append(len(after_sla_translation) == (after_sla_code['n_sla_codes'].values[0] * len(after_date_category)))

### Fifth before test

In [93]:
timing.append('after')

In [94]:
descriptions.append('''Test the uniqueness of the rows in tbl_ref_sla_translation''')

In [95]:
after_sla_translation_nodup = after_sla_translation.drop_duplicates()

In [96]:
results.append(len(after_sla_translation) == len(after_sla_translation_nodup))

### Sixth after test

In [97]:
timing.append('after')

In [98]:
descriptions.append(
    '''Test that all sla_ids, sla_codes and date_category_ids have been \
    accounted for in tbl_ref_sla_translation''')

In [99]:
after_unique_sla_translation = after_sla_translation.nunique()

In [100]:
results.append(
    ((after_unique_sla_translation['sla_id'] == after_count_sla_id)
     and (after_unique_sla_translation['date_category_id'] == len(
         after_date_category)) and (after_unique_sla_translation['sla_code'] ==
                                    after_sla_code['n_sla_codes'].values[0])))

### Seventh after test

In [97]:
timing.append('after')

In [ ]:
descriptions.append(
    '''Test that the number of rows insrted into tbl_ref_sla_code is equal to (ninserts + n_orig)**2 -(n_orig)**2'''
)

In [122]:
before_sla_code.max_sla_code.values[0]

64

In [124]:
after_sla_code

,min_sla_code,max_sla_code,n_sla_codes
0,1,64,64


In [127]:
after_count_sla_id

8

In [102]:
descriptions.append(
    '''Test that after removing original rows from tbl_ref_sla_trnaslation there are (m+n)**2 -(n)**2 rows leftover'''
)

In [103]:
after_sla_translation

,sla_id,sla_code,date_category_id
0,A,1,1
1,A,2,1
2,A,3,1
3,A,4,1
4,A,17,1
...,...,...,...
123,T,28,2
124,T,30,2
125,T,36,2
126,T,47,2


In [104]:
before_sla_translation

,sla_id,sla_code,date_category_id
0,A,1,1
1,A,2,1
2,A,3,1
3,A,4,1
4,A,17,1
...,...,...,...
67,T,20,2
68,T,22,2
69,T,28,2
70,T,30,2
